# Task 1 - EDA (Explaratory Data Analysis)

## Subtask 1 - Connection to DB

Here we import the necessary modules, as well as a custom connection to database function from src/commonfunctions.py 

Purpose is to improve code readbility and reusability throughout the assessment. 

In [ ]:
import pandas as pd
from src.commonfunctions import ConnectToDB

Connecting to the Database in data\survive.db and then executing the query and saving the results to a dataframe in pandas.

In [2]:
pathtodb = 'data\survive.db'

df = pd.read_sql_query("SELECT * from survive", ConnectToDB(pathtodb))

# Verify that result of SQL query is stored in the dataframe
df


Database created and Successfully Connected to SQLite
SQLite Database Version is:  [('3.36.0',)]


,ID,Survive,Gender,Smoke,Diabetes,Age,Ejection Fraction,Sodium,Creatinine,Pletelets,Creatinine phosphokinase,Blood Pressure,Hemoglobin,Height,Weight,Favorite color
0,TIG1GE,0,Male,Yes,Normal,50,Low,141,0.7,266000.0,185,105,12.3,180,93,green
1,VXUCV9,1,Male,No,Normal,75,Low,134,2.5,224000.0,99,162,13.0,140,47,black
2,AFH4F1,1,Female,No,Pre-diabetes,80,Low,135,1.3,192000.0,776,179,12.4,150,61,white
3,ZK6PAC,0,Male,No,Normal,-60,Low,136,0.9,228000.0,2261,136,15.3,130,48,yellow
4,6CVBRT,0,Female,No,Normal,70,Low,141,0.9,358000.0,1202,171,13.3,177,64,blue
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14995,91MBFV,0,Male,Yes,Normal,50,Low,140,0.9,362000.0,298,109,11.8,178,51,blue
14996,3BK9WM,1,Male,Yes,Normal,62,Low,140,0.9,253000.0,231,141,14.6,171,114,green
14997,01ZP6F,0,Female,No,Normal,61,Low,141,0.9,229000.0,84,173,9.2,156,71,blue
14998,JMSGEG,0,Male,No,Normal,77,Low,137,1.1,406000.0,109,154,11.3,130,32,red


## Subtask 2 - Initial EDA

16 columns with 15000 rows are recorded as the shape of the df, also platelets are wrongly spelt? We see below that there are no rows with missing values except for the Creatinine column. 

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 16 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   ID                        15000 non-null  object 
 1   Survive                   15000 non-null  object 
 2   Gender                    15000 non-null  object 
 3   Smoke                     15000 non-null  object 
 4   Diabetes                  15000 non-null  object 
 5   Age                       15000 non-null  int64  
 6   Ejection Fraction         15000 non-null  object 
 7   Sodium                    15000 non-null  int64  
 8   Creatinine                14501 non-null  float64
 9   Pletelets                 15000 non-null  float64
 10  Creatinine phosphokinase  15000 non-null  int64  
 11  Blood Pressure            15000 non-null  int64  
 12  Hemoglobin                15000 non-null  float64
 13  Height                    15000 non-null  int64  
 14  Weight

This is where the real trouble starts, we find that the columns contain dubious and ambigious data, for eg. Survive column has 4 values when it should be a boolean value and ID contains only 14042 unique entries, which means some IDs have been duplicated. Below we also write a log of the summary of data in data/Dataset_summary.txt for your reference. Please head over to that section to find out more detailed summary. 

In [10]:
df.nunique()

ID                          14042
Survive                         4
Gender                          2
Smoke                           4
Diabetes                        3
Age                            87
Ejection Fraction               5
Sodium                         27
Creatinine                     40
Pletelets                     176
Creatinine phosphokinase      208
Blood Pressure                139
Hemoglobin                     86
Height                         60
Weight                        123
Favorite color                  6
dtype: int64

In [8]:
file = open("data/Dataset_Summary.txt","r+")
file.truncate(0)
file.close()

for column in df:
    file = open("data/Dataset_Summary.txt","a")
    file.write(str(df[column].value_counts()))
    file.write('\n')
    file.write('='*20)
    file.write('\n')
    file.close()

## Subtask 3 - Data Cleaning

ETL Pipeline #1
Going sequentially, column by columns we start with the first column for ID - i will assume that any subsequent entry with the same ID is an error and treat it as such. 

In [70]:
cleaned_df = df.copy(deep= True)
#ETL Pipeline Number 1 
cleaned_df.drop_duplicates(subset ="ID", keep = 'first', inplace = True) 

ETL Pipeline #2 Survive Column contains mix of 0 and 1 boolean and Yes and No 

In [71]:
cleaned_df.Survive = cleaned_df.Survive.map({'Yes':1, 'No':0, '1':1,'0':0})


ETL Pipeline #3 Change Gender to Encoded Boolean Values 

In [72]:
cleaned_df.Gender = cleaned_df.Gender.map({'Male':1,'Female':0})

ETL Pipeline #4 Change Smoke cat to all upper case then boolean encode yes and no values

In [73]:
cleaned_df['Smoke'] = cleaned_df['Smoke'].str.upper()  

cleaned_df.Smoke = cleaned_df.Smoke.map({'YES':1,'NO':0})

ETL Pipeline #5 Remove Erranous Age values for eg. Negative ages and ages > 100 

In [74]:
cleaned_df = cleaned_df[cleaned_df.Age >0]  
cleaned_df = cleaned_df[cleaned_df.Age < 100 ]

ETL Pipeline #6 Changing Ejection Fraction to Low High and Normal

In [75]:
cleaned_df['Ejection Fraction'] = cleaned_df['Ejection Fraction'].map({'Low':'Low','Normal':'Normal','High':'High','L':'Low','N':'Normal'})

ETL Pipeline #7 Filling in missing values for Creatinine 

In [76]:
cleaned_df = cleaned_df.fillna(method="ffill")

In [80]:
cleaned_df.to_csv(r'data/Cleaned_Data.csv')

## Subtask 4 - Data Visualizations 


Interactive Data Visualizations 